### CB數據全部重新更新

徹底重新執行數據，重新跑過一輪，時間會花費較久

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests, os, time

def download_cb_reports_month(year="2024", month="7"):
    # 民國年資料夾：e.g., 2024 ➜ 113年
    roc_year = str(int(year) - 1911)
    # 路徑可以修改
    folder = os.path.join("U:/CB數據庫/cb_reports", f"{roc_year}年")
    os.makedirs(folder, exist_ok=True)

    options = webdriver.ChromeOptions()
    # 如需除錯可註解 headless
    options.add_argument("--headless")
    driver = webdriver.Chrome(service=Service(), options=options)
    wait = WebDriverWait(driver, 10)

    # 開啟目標網頁
    driver.get("https://www.tpex.org.tw/zh-tw/bond/info/statistics-cb/day.html")

    try:
        # 等待所有選單元素
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.select-year.selectobj")))
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.select-month.selectobj")))
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "select[name='fileCode']")))

        # 每次都重新定位選單元素（避免舊元素失效）
        year_select = Select(driver.find_element(By.CSS_SELECTOR, "select.select-year.selectobj"))
        month_select = Select(driver.find_element(By.CSS_SELECTOR, "select.select-month.selectobj"))
        report_select = Select(driver.find_element(By.CSS_SELECTOR, "select[name='fileCode']"))

        year_select.select_by_value(year)
        month_select.select_by_value(str(month))
        report_select.select_by_value("rsta0113")

        # 等待頁面更新
        time.sleep(2)
        wait.until(EC.presence_of_element_located((By.XPATH, "//a[contains(@href, '.csv')]")))

        # 擷取表格
        rows = driver.find_elements(By.XPATH, '//table//tr')[1:]  # 跳過表頭
        for row in rows:
            try:
                tds = row.find_elements(By.TAG_NAME, "td")
                if len(tds) >= 2:
                    date_str = tds[0].text.strip().replace("/", "")
                    link = tds[1].find_element(By.TAG_NAME, "a").get_attribute("href")
                    filepath = os.path.join(folder, f"{date_str}.csv")

                    if os.path.exists(filepath):
                        print(f"{date_str}.csv 已存在，略過")
                        continue

                    res = requests.get(link)
                    if res.status_code == 200:
                        with open(filepath, "wb") as f:
                            f.write(res.content)
                        print(f"{date_str}.csv ✅ 成功下載")
                    else:
                        print(f"{date_str}.csv ❌ 下載失敗")
            except Exception as e:
                print(f"⚠️ 跳過一列：{e}")

    except Exception as e:
        print(f"❌ 無法處理 {year} 年 {month} 月：{e}")

    driver.quit()
    print(f"📦 完成：{year} 年 {month} 月資料已儲存至 {folder}")

# 🧪 呼叫單月下載
if __name__ == "__main__":
    from datetime import datetime

    # 取得今天日期
    today = datetime.today()

    # 自動設定西元年與月份
    year = str(today.year)
    month = str(today.month)

    # 執行當月下載
    download_cb_reports_month(year, month)


<>:11: SyntaxWarning: invalid escape sequence '\C'
<>:11: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Psc.PT6202\AppData\Local\Temp\ipykernel_8916\577820453.py:11: SyntaxWarning: invalid escape sequence '\C'
  folder = os.path.join("U:\CB數據庫\cb_reports", f"{roc_year}年")


1140827.csv ✅ 成功下載
1140826.csv ✅ 成功下載
1140825.csv ✅ 成功下載
1140822.csv ✅ 成功下載
1140821.csv ✅ 成功下載
1140820.csv ✅ 成功下載
1140819.csv ✅ 成功下載
1140818.csv ✅ 成功下載
1140815.csv ✅ 成功下載
1140814.csv ✅ 成功下載
1140813.csv ✅ 成功下載
1140812.csv ✅ 成功下載
1140811.csv ✅ 成功下載
1140808.csv ✅ 成功下載
1140807.csv ✅ 成功下載
1140806.csv ✅ 成功下載
1140805.csv ✅ 成功下載
1140804.csv ✅ 成功下載
1140801.csv ✅ 成功下載
📦 完成：2025 年 8 月資料已儲存至 U:\CB數據庫\cb_reports\114年


問題點：只會跑當月的，這樣跨月才更新會完蛋

協助處理民國96年至114年之數據，轉化成開高低收四個數據庫資料，且儲存成csv與xlsx兩種檔案型式

In [ ]:
import pandas as pd
import os

def safe_read_cp950(file_path):
    """使用 cp950 編碼讀取 CSV，跳過前3行與後3行之表頭與備註"""
    try:
        df = pd.read_csv(file_path, encoding="cp950", skiprows=3)
        df = df[:-3] if len(df) > 3 else df
        df = df[df["交易"] == "等價"]
        return df
    
    # 如果讀取失敗，回傳空的 DataFrame
    except Exception as e:
        print(f"❌ {os.path.basename(file_path)} 讀取失敗：{e}")
        return pd.DataFrame()

def normalize_roc_date(raw_str):
    """將100年以內數據前面補0處理，並轉為 datetime"""
    if len(raw_str) == 6 and raw_str.isdigit():
        raw_str = "0" + raw_str
    elif len(raw_str) != 7 or not raw_str.isdigit():
        return pd.NaT
    try:
        roc_year = int(raw_str[:3])
        year = roc_year + 1911
        month = int(raw_str[3:5])
        day = int(raw_str[5:7])
        return pd.Timestamp(year=year, month=month, day=day)
    except:
        return pd.NaT

def convert_cb_years_to_refined_matrix(start_year, end_year, base, base_name):
    """跨年度整併所有 CSV 成為代號 / 標的 / 收盤價 欄位層級格式，並輸出為 xlsx 和 csv"""
    base_folder = "U:/CB數據庫/cb_reports"
    dfs = []
    column_order = [] # 紀錄所有商品之欄位順序(代號+名稱)
    
    # 處理每一年之資料夾
    for year in range(int(start_year), int(end_year) + 1):
        folder_path = os.path.join(base_folder, f"{year}年")
        print(f"\n📅 處理民國 {year} 年資料中...")
        
        # 處理每天之資料
        for file in sorted(os.listdir(folder_path)):
            if not file.endswith(".csv"):
                continue

            file_path = os.path.join(folder_path, file)
            raw_date_str = file.replace(".csv", "")
            date = normalize_roc_date(raw_date_str)
            if pd.isna(date):
                print(f"⚠️ 日期格式錯誤：{file}，略過")
                continue

            # 讀取與清洗資料
            df = safe_read_cp950(file_path)
            if df.empty:
                continue

            code_col = next((c for c in df.columns if "代號" in c), None)
            name_col = next((c for c in df.columns if "名稱" in c or "債券名稱" in c), None)
            close_col = next((c for c in df.columns if base in c), None)

            if not all([code_col, name_col, close_col]):
                print(f"⚠️ 欄位不完整：{file}，略過")
                continue

            # 建立清洗後的資料表
            sub_df = df[[code_col, name_col, close_col]].dropna()
            sub_df.columns = ["code", "name", "close"]
            sub_df["date"] = date
            dfs.append(sub_df)

            # 建立欄位順序，依照商品首次出現順序排列
            for code, name in zip(sub_df["code"], sub_df["name"]):
                if (code, name) not in column_order:
                    column_order.append((code, name))

    if dfs:
        # 合併所有資料
        combined_df = pd.concat(dfs, ignore_index=True)

        # 建立代號 → 最新標的名稱對照表（以最新日期為準）
        latest_name_map = (
            combined_df.sort_values("date")
            .drop_duplicates(subset=["code"], keep="last")
            .set_index("code")["name"]
        )

        # 將所有標的名稱更新為最新名稱
        combined_df["name"] = combined_df["code"].map(latest_name_map)

        # 建立三層矩陣：index 為日期，columns 為代號＋標的
        pivot_df = combined_df.pivot_table(
            index="date", columns=["code", "name"], values="close", aggfunc="first"
        )

        pivot_df.columns.names = ["代號", "標的"]

        # 根據每個商品首次出現的時間排序欄位
        first_appearance_df = (
            combined_df.groupby(["code", "name"])["date"]
            .min()
            .reset_index()
            .rename(columns={"date": "first_date"})
        )

        # 依照首次出現時間 → 代號排序
        sorted_columns = (
            first_appearance_df.sort_values(["first_date", "code"])
            .set_index(["code", "name"])
            .index
        )

        # 重新排序欄位
        pivot_df = pivot_df[sorted_columns]

        # 儲存為 CSV
        csv_path = os.path.join(base_folder, base_name + ".csv")

        pivot_df.to_csv(csv_path, encoding="utf-8-sig")

        print(f"\n✅ 完成整併並儲存： CSV：{csv_path}")
        return pivot_df
    else:
        print("\n🚫 沒有任何有效資料")
        return pd.DataFrame()

# ✨ 執行：一次處理 96～114 年
if __name__ == "__main__":
    convert_cb_years_to_refined_matrix("96", "114", base = "收市", base_name = "收盤價")
    convert_cb_years_to_refined_matrix("96", "114", base = "開市", base_name = "開盤價")
    convert_cb_years_to_refined_matrix("96", "114", base = "最高", base_name = "最高價")
    convert_cb_years_to_refined_matrix("96", "114", base = "最低", base_name = "最低價")
    convert_cb_years_to_refined_matrix("96", "114", base = "單位", base_name = "單位")
    convert_cb_years_to_refined_matrix("96", "114", base = "漲跌", base_name = "漲跌")
    convert_cb_years_to_refined_matrix("96", "114", base = "金額", base_name = "成交金額")

<>:34: SyntaxWarning: invalid escape sequence '\C'
<>:34: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Psc.PT6202\AppData\Local\Temp\ipykernel_8916\933368682.py:34: SyntaxWarning: invalid escape sequence '\C'
  base_folder = "U:\CB數據庫\cb_reports"



📅 處理民國 96 年資料中...

📅 處理民國 97 年資料中...

📅 處理民國 98 年資料中...

📅 處理民國 99 年資料中...

📅 處理民國 100 年資料中...

📅 處理民國 101 年資料中...

📅 處理民國 102 年資料中...

📅 處理民國 103 年資料中...

📅 處理民國 104 年資料中...

📅 處理民國 105 年資料中...

📅 處理民國 106 年資料中...

📅 處理民國 107 年資料中...

📅 處理民國 108 年資料中...

📅 處理民國 109 年資料中...

📅 處理民國 110 年資料中...

📅 處理民國 111 年資料中...

📅 處理民國 112 年資料中...

📅 處理民國 113 年資料中...

📅 處理民國 114 年資料中...


C:\Users\Psc.PT6202\AppData\Local\Temp\ipykernel_8916\933368682.py:81: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(dfs, ignore_index=True)



✅ 完成整併並儲存： CSV：U:\CB數據庫\cb_reports\收盤價1.csv


In [ ]:
import pandas as pd
import numpy as np
import os

def insert_copy_column_and_save(filepath):
    """讀取 CSV，插入複製欄並儲存為不含欄位名稱與 index 的修正 CSV"""
    
    # 讀取原始 CSV（無欄位名稱）
    try:
        df = pd.read_csv(filepath, header=None)
    except Exception as e:
        print(f"無法讀取檔案：{filepath}，錯誤：{e}")
        return

    # 取得檔名與資料夾
    file_name = os.path.splitext(os.path.basename(filepath))[0]
    output_folder = os.path.dirname(filepath)

    # 插入複製欄在第 0 欄右側
    df = df.copy()
    df.insert(1, '複製欄', df.iloc[:, 0])

    # 第 0 欄：將「代號」「標的」改為 NaN
    df.iloc[0:2, 0] = np.nan

    # 複製欄：將「date」與所有日期列改為 NaN
    df.iloc[2:, 1] = np.nan

    # 複製欄：第 3 格填入檔名（如「收盤價」「開盤價」「單位」等）
    df.iloc[2, 1] = file_name

    # 儲存為 CSV（無 index、無欄位名稱）
    output_path = os.path.join(output_folder, f"{file_name}_修正.csv")
    df.to_csv(output_path, index=False, header=False, encoding="utf-8-sig")


def clean_and_save_cb_report(report_type, input_dir, output_dir):
    """
    清理 CB 報表並儲存為好讀取的 CSV 檔案。
    
    參數：
    - report_type: 報表類型（如 "開盤價", "最高價"）
    - input_dir: 原始 CSV 檔案所在資料夾
    - output_dir: 清理後 CSV 檔案儲存資料夾
    """
    filename = f"{report_type}_修正.csv"
    input_path = os.path.join(input_dir, filename)

    # 讀取原始 CSV（不使用欄位名稱）
    df_raw = pd.read_csv(input_path, encoding="utf-8-sig", header=None, low_memory=False)

    # 第 0 列是代號，第 1 列是標的名稱
    codes = df_raw.iloc[0]
    names = df_raw.iloc[1]

    # 合併代號與標的名稱作為欄位名稱
    combined_columns = [
        f"{str(code).strip()} {str(name).strip()}"
        if pd.notna(code) and pd.notna(name) else ""
        for code, name in zip(codes, names)
    ]

    # 移除前 3 列（代號、標的、收盤價文字列） 
    df_cleaned = df_raw.iloc[3:].copy() 
    df_cleaned.columns = combined_columns 
    
    # 將第一欄（日期）設為 index 
    df_cleaned = df_cleaned.set_index(df_cleaned.columns[0]) 
    df_cleaned.index.name = "日期"
    df_cleaned = df_cleaned.drop(columns=["代號 標的"], errors="ignore") 
    
    # 將 index 轉為 datetime，並移除無法解析的日期 
    df_cleaned.index = pd.to_datetime(df_cleaned.index, format="%Y-%m-%d", errors="coerce") 
    df_cleaned = df_cleaned[~df_cleaned.index.isna()]

    # 儲存清理後的 CSV
    output_path = os.path.join(output_dir, f"CB_{report_type}.csv")
    df_cleaned.to_csv(output_path, encoding="utf-8-sig")

    return df_cleaned

insert_copy_column_and_save(r"U:/CB數據庫/cb_reports/收盤價.csv")
insert_copy_column_and_save(r"U:/CB數據庫/cb_reports/開盤價.csv")
insert_copy_column_and_save(r"U:/CB數據庫/cb_reports/最高價.csv")
insert_copy_column_and_save(r"U:/CB數據庫/cb_reports/最低價.csv")
insert_copy_column_and_save(r"U:/CB數據庫/cb_reports/單位.csv")
insert_copy_column_and_save(r"U:/CB數據庫/cb_reports/成交金額.csv")

# 使用clean_and_save_cb_report函數
input_folder = r"U:/CB數據庫/cb_reports"
output_folder = r"U:/CB數據庫"
clean_and_save_cb_report("開盤價", input_folder, output_folder)
clean_and_save_cb_report("最高價", input_folder, output_folder)
clean_and_save_cb_report("最低價", input_folder, output_folder)
clean_and_save_cb_report("收盤價", input_folder, output_folder)
clean_and_save_cb_report("單位", input_folder, output_folder)
clean_and_save_cb_report("成交金額", input_folder, output_folder)

C:\Users\Psc.PT6202\AppData\Local\Temp\ipykernel_8916\4140753109.py:10: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,2

14772 聚陽二       15221 堤維一       15291 樂士一       15362 和大二  \
日期                                                                           
2007-01-02  1,245,000       952,000         579,900         4,199,700        
2007-01-03             NaN  3,331,200       96,500          20,745,200       
2007-01-04             NaN  1,422,750                  NaN  2,671,750        
2007-01-05             NaN  2,622,050       96,500          367,000          
2007-01-08             NaN  121,400         281,750         3,702,550        
...                    ...             ...             ...             ...   
2025-08-21             NaN             NaN             NaN             NaN   
2025-08-22             NaN             NaN             NaN             NaN   
2025-08-25             NaN             NaN             NaN             NaN   
2025-08-26             NaN             NaN             NaN             NaN   
2025-08-27             NaN             NaN             NaN             NaN   

                 15691 濱川一      16091 大亞1A       16171 榮星一       17851 光洋一  \
日期                                                                           
2007-01-02  672,400         910,000         494,000         16,215,500       
2007-01-03  112,500                    NaN  629,800         234,650          
2007-01-04  219,500         1,720,000       1,107,150       26,790,850       
2007-01-05             NaN             NaN  365,200         13,043,400       
2007-01-08  777,000         700,000         123,800         2,286,050        
...                    ...             ...             ...             ...   
2025-08-21             NaN             NaN             NaN             NaN   
2025-08-22             NaN             NaN             NaN             NaN   
2025-08-25             NaN             NaN             NaN             NaN   
2025-08-26             NaN             NaN             NaN             NaN   
2025-08-27             NaN             NaN             NaN             NaN   

                 19022 台紙二       20062 東鋼二  ...       31312 弘塑二  \
日期                                          ...                   
2007-01-02  262,000         3,653,100       ...             NaN   
2007-01-03             NaN  3,988,950       ...             NaN   
2007-01-04             NaN  721,400         ...             NaN   
2007-01-05             NaN  135,650         ...             NaN   
2007-01-08             NaN  989,900         ...             NaN   
...                    ...             ...  ...             ...   
2025-08-21             NaN             NaN  ...  28,939,450       
2025-08-22             NaN             NaN  ...  13,761,950       
2025-08-25             NaN             NaN  ...  7,650,000        
2025-08-26             NaN             NaN  ...  3,912,150        
2025-08-27             NaN             NaN  ...  15,376,700       

                 26418 正德八       31672 大量二     75561 意德士一永   45811 光隆精密一KY  \
日期                                                                           
2007-01-02             NaN             NaN             NaN             NaN   
2007-01-03             NaN             NaN             NaN             NaN   
2007-01-04             NaN             NaN             NaN             NaN   
2007-01-05             NaN             NaN             NaN             NaN   
2007-01-08             NaN             NaN             NaN             NaN   
...                    ...             ...             ...             ...   
2025-08-21  10,828,700      77,879,550      7,168,850       61,394,200       
2025-08-22  5,272,750       42,996,750      1,314,800       146,922,700      
2025-08-25  2,722,100       29,609,650      120,000         32,196,350       
2025-08-26  9,991,300       13,978,300      2,101,000       27,292,350       
2025-08-27  4,016,850       13,284,250      5,575,650       13,822,750       

                 68211 聯寶一      75562 意德士二       37075 漢磊五       62075 雷科五  \
日期                    